In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DeepLearning/Yolov6
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
!pip install -r requirements.txt

/content/drive/MyDrive/DeepLearning/Yolov6
Cloning into 'YOLOv6'...
remote: Enumerating objects: 3838, done.
remote: Total 3838 (delta 0), reused 0 (delta 0), pack-reused 3838 (from 1)
Receiving objects: 100% (3838/3838), 43.99 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (2297/2297), done.
Updating files: 100% (232/232), done.
/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.1 MB/s eta 0:00:00


In [ ]:
!python tools/train.py --batch 32 --conf configs/yolov6n_finetune.py --epochs 100 --img-size 600 --data /content/drive/MyDrive/DeepLearning/Dataset/mydataset.yaml --device 0

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  with amp.autocast(enabled=self.device != 'cpu'):
/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6/yolov6/models/losses/loss.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
     53/99   0.001704    0.6013         0    0.7656:  43%|████▎     | 10/23 [00:12<00:20,  1.58s/it]/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6/yolov6/core/engine.py:150: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=self.device != 'cpu'):
/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6/yolov6/models/losses/loss.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
     53/99   0.001704     0.599  

In [ ]:
%cd /content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from yolov6.utils.events import LOGGER, load_yaml
from yolov6.layers.common import DetectBackend
from yolov6.data.data_augment import letterbox
from yolov6.utils.nms import non_max_suppression

checkpoint = "/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6/runs/train/exp/weights/best_ckpt"  # Path to custom weights
device = "cpu" 
half = False 
img_size = [608, 608] 

if not os.path.exists(f"{checkpoint}.pt"):
    raise FileNotFoundError(f"Checkpoint file not found: {checkpoint}.pt")
device = torch.device('cuda:0' if device == 'cuda' and torch.cuda.is_available() else 'cpu')

def process_image(path, img_size, stride, half):
    try:
        img_src = cv2.imread(path)
        if img_src is None:
            raise FileNotFoundError(f"Invalid image file or unsupported format: {path}")
    except Exception as e:
        LOGGER.warning(e)
        raise e

    image = letterbox(img_src, img_size, stride=stride)[0]

    image = image.transpose((2, 0, 1))  
    image = torch.from_numpy(np.ascontiguousarray(image))
    image = image.half() if half else image.float()  #
    image /= 255.0  

    return image, img_src

model = DetectBackend(f"{checkpoint}.pt", device=device)
stride = model.stride

if half and device.type != 'cpu':
    model.model.half()
else:
    model.model.float()
    half = False

if device.type != 'cpu':
    model(torch.zeros(1, 3, *img_size).to(device).type_as(next(model.model.parameters()))) 

image_path = "/content/drive/MyDrive/DeepLearning/Dataset/test/images/airport_77_jpg.rf.e1a1a14a43f962dc16d5c66fffd1ada6.jpg"

if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image not found at {image_path}")

try:
    image, original_img = process_image(image_path, img_size, stride, half)
    image = image.unsqueeze(0).to(device)

    predictions = model(image)
    predictions = non_max_suppression(predictions, conf_thres=0.3, iou_thres=0.45)[0] 

    if predictions is not None:
        for det in predictions:
            x1, y1, x2, y2, conf = int(det[0]), int(det[1]), int(det[2]), int(det[3]), det[4].item() 
            cv2.rectangle(original_img, (x1-6, y1-4), (x2-4, y2-3), (0, 0, 255), 1)
            cv2.putText(original_img, f'{conf:.2f}', (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  

    
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

except Exception as e:
    print(f"Error during inference: {e}")


Namespace(weights='/content/drive/MyDrive/DeepLearning/Yolov6/runs/train/exp/weights/best_ckpt.pt', source='/content/drive/MyDrive/DeepLearning/Dataset/train/images/airport_102_jpg.rf.0bc87c7c57f68d9a0acb281b532ce2f8.jpg', webcam=False, webcam_addr='0', yaml='/content/drive/MyDrive/DeepLearning/Dataset/mydataset.yaml', img_size=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device='0', save_txt=True, not_save_img=False, save_dir='/content/drive/MyDrive/DeepLearning/Yolov6/results', view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='exp', hide_labels=False, hide_conf=False, half=False)
checkpoint best_ckpt.pt not exist, try to downloaded it from github.
downloading url is: https://github.com/meituan/YOLOv6/releases/download/0.4.0/best_ckpt.pt, pealse make sure the version of the downloading model is correspoing to the code version!
Traceback (most recent call last):
  File "/content/drive/MyDrive/DeepLearning/Yolov6/YOLOv6/tools/infer.py", line